<a href="https://colab.research.google.com/github/CelioSGA/LBS_TRACKER/blob/main/LBS_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q -U google-generativeai
!pip install -q --upgrade generativeai

In [1]:
!pip install opencellid

  Preparing metadata (setup.py) ... done
  Created wheel for opencellid: filename=opencellid-1.3.4-py3-none-any.whl size=8838 sha256=03e29358823359ca5ca44dd20accfdae495d9a5c3b5fa5c56fda19ed564d858b
  Stored in directory: /root/.cache/pip/wheels/fd/6a/7c/bf6beef4c920caed0562af986c336d5031fcd98be4b7d01f97
Successfully built opencellid


In [8]:
import pathlib
import textwrap
import json
from datetime import datetime
import pytz

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

import pandas as pd
filename = "724.csv"
headers = ['radio', 'mcc', 'mnc', 'lac', 'cellid', 'check', 'longitude', 'latitude', 'range', 'samples', 'changeable1', 'created', 'updated', 'signal']

data = pd.read_csv(filename, header = None)
data.columns = headers

print("data.info()")  # Shows the first few rows
print(data.info())  # Displays DataFrame information

import google.generativeai as genai

from google.colab import userdata
API_KEY=userdata.get('API_KEY')

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def hex2dec(hex_string):
    """Converts a hexadecimal string to a decimal number."""
    decimal_value = 0
    base = 1
    for i in range(len(hex_string) - 1, -1, -1):
        digit = hex_string[i]
        if digit.isdigit():
            decimal_value += int(digit) * base
        elif digit.isalpha():
            decimal_value += (ord(digit) - ord('A') + 10) * base
        base *= 16
    return decimal_value

genai.configure(api_key=API_KEY)

#estudar melhor
generation_config = {
    "candidate_count": 1,
    "temperature" : 0.7,
    "top_k" : 40,
    "top_p" : 0.95
}

#estudar melhor
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "HATE" : "BLOCK_NONE",
    "DANGEROUS" : "BLOCK_NONE"
}
model = genai.GenerativeModel(model_name = "gemini-1.5-pro-latest",
                              generation_config = generation_config,
                              safety_settings=safety_settings)

#iniciar modelo de acordo com aplicação este é o modelo Alfred

now = datetime.now(pytz.timezone('America/Sao_Paulo'))
local_time = now.strftime('Hoje é %A, %d/%m/%Y, são %H:%M')

print(local_time)
print()

history = [] #carregar history do usuário (Gdrive ou BD) (LIMITE?)
chat = model.start_chat(history = history)

start_data = local_time + '''
Olá Gemini,

Você é uma plataformade localização de dispositivos, reastreadores de veículos, cargar, pessoas, animais, etc.
Este modelo de AI utiliza torres celulares (LBS) para a lozalização.

Muitas vezes os sinais de GPS ficam indisponíveis e é necessário um método alternativo.
Informme as coordenadas em texto e crie um link para o Google maps com a latitude e longitude informados.
Modelo do link: https://www.google.com/maps/search/?api=1&query=<latitude>,<longitude>

Não é necessário que a localização seja exata, quilometros de desvio são toleráveis.
Dê me informações detalhadas do como você realizaou o processo.
O dispositivo não possui GPS, apenas modem GSM/LTE e WiFi.
A última localização dele foi:
**Latitude:** -23.58365
**Longitude:** -46.64256
Considere a localização acima e use as torres desta localização.

Sempre que necessário procure em sua base de dados ou no Google informações de longitude, latitude, etc. para localidades, bairros, cidades, etc.
A seguir mandarei as torres encontradas. O campo created e updated estão em UNIXTIME dados mais recentes NÃO têm prioridade.
Se não for informado ignore RSSI.

Mostre-me um resultado mesmo que impreciso ou desatualizado.
Aguarde até que eu envie a lista de torres e mostre-me o resultado. O formato da lista é JSON.
Utilize os dados abaixo e usando os campos Latitude e Longitue, e faça os cáculos utilizando o método de triangulação.
'''

response = chat.send_message(start_data);
print("Traker: ", response.text,"\n")

torres = [
    {
        "cell": 1,
        "bcch": 240,
        "rssi": 21,
        "bsic": 42,
        "cellid": "5AE5",
        "mcc": 724,
        "mnc": 5,
        "lac": "1A9B"
    },
    {
        "cell": 2,
        "bcch": 662,
        "rssi": 9,
        "bsic": 38,
        "cellid": "2E58",
        "mcc": 724,
        "mnc": 5,
        "lac": "1A9B"
    },
    {
        "cell": 3,
        "bcch": 251,
        "rssi": 9,
        "bsic": 8,
        "cellid": "5AE3",
        "mcc": 724,
        "mnc": 5,
        "lac": "1A9B"
    },
    {
        "cell": 4,
        "bcch": 586,
        "rssi": 8,
        "bsic": 0,
        "cellid": "2DF4",
        "mcc": 724,
        "mnc": 5,
        "lac": "1A9B"
    }
]

counter = 1
tower_list = []
counter = 0
for torre in torres:
    # Extraia os campos mnc, mcc e cellid de torres
    mnc = torre["mnc"]
    mcc = torre["mcc"]
    cellid = hex2dec(torre["cellid"])
    lac = hex2dec(torre["lac"])
    result = data[(data["mnc"] == mnc) & (data["mcc"] == mcc) & ( (data["lac"] == lac) | (data["cellid"] == cellid) )] #& (data["cellid"] == cellid) &# (data["lac"] == lac)

    for index, r in result.iterrows():
      tower_list.append(r.to_dict())

    print()
    counter += 1
tower_list = json.dumps(tower_list, indent=2)
print(tower_list)
response = chat.send_message(tower_list);
print("Traker: ", response.text,"\n")

prompt = ""
while prompt != "fim" :
  prompt = input("Prompt: ")

  if prompt == "fim": break
  if prompt == "end": break
  if prompt == "exit": break

  response = chat.send_message(prompt)
  print("Tracker: ", response.text,"\n")

print(chat.history) #exibe hitórico para análise // salvar em BD/Drive?



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive
data.info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873637 entries, 0 to 1873636
Data columns (total 14 columns):
 #   Column       Dtype  
---  ------       -----  
 0   radio        object 
 1   mcc          int64  
 2   mnc          int64  
 3   lac          int64  
 4   cellid       int64  
 5   check        int64  
 6   longitude    float64
 7   latitude     float64
 8   range        int64  
 9   samples      int64  
 10  changeable1  int64  
 11  created      int64  
 12  updated      int64  
 13  signal       int64  
dtypes: float64(2), int64(11), object(1)
memory usage: 200.1+ MB
None
Hoje é Tuesday, 14/05/2024, são 15:22

Traker:  Entendi! Aguardo você me enviar a lista de torres no formato JSON. Com base nas coordenadas que você forneceu (-23.58365, -46.64256) e nas informações das torres, farei o possível para  es